In [50]:
# SCS_3547_006 Intelligent Agents
# Ben Pierce
# 2020-07-12
# Assignment 3: Monte Carlo Policy Optimization

In [51]:
import IPython
from enum import IntEnum
import random

In [52]:
# Enum of all the possible actions our agent can make in the simulation.
class Action(IntEnum):
    Up = 1,
    Down = 2,
    Left = 3,
    Right = 4

# Represents a single cell in the gridworld. 
class Cell(object):
  def __init__(self, row, col, index, terminal_cell, available_actions = None):
    self.row = row
    self.col = col
    self.value = 0
    self.index = index
    self.terminal_cell = terminal_cell
    self.available_actions = available_actions

  def __str__(self):
        return 'Cell ({0}:{1})'.format(self.row, self.col)

  __repr__ = __str__

  def __eq__(self, other):
      if type(other) is type(self):
          return (self.row == other.row and self.col == other.col) 
      else:
          return False

# A grid composed of a number of cells and helper methods   
class Grid(object):
  def __init__(self, rows, cols, terminal_cells):    
    self.rows = rows  # Number of rows in the gridworld
    self.cols = cols  # Number of columns in the gridworld                
    self.terminal_cells = terminal_cells  # A list of the terminal cells    
    self.init_grid()  # Initialize gridworld

  # Initializer for the grid
  def init_grid(self):
    idx = 0
    cells = []  # List of cells in this gridworld

    # For each row and column in this gridworld
    for r in range(1, self.rows + 1):
      for c in range(1, self.cols + 1):
        terminal = False
        for terminal_cell in self.terminal_cells:
          if terminal_cell.index == idx:
            terminal = True 
            break 

        # Calculate the actions available from this cell. 
        actions = []
        if not terminal:
          if r > 1:
            actions.append(Action.Up)
          if r < self.rows:
            actions.append(Action.Down)
          if c > 1:
            actions.append(Action.Left)
          if c < self.cols:
            actions.append(Action.Right)

        cells.append(Cell(r, c, idx, terminal, actions))
        idx += 1        
    self.cells = cells    

  # Helper method to get a cell based on a row and col index
  def get_cell(self, row, col):
    idx = ((row - 1) * self.cols) + col - 1
    return self.cells[idx]

  # Gets the best actions for a cell, based on the highest value of a cell's neighbors
  def get_cell_best_actions(self, row, col):
    best_actions = []
    cell = self.get_cell(row, col)

    if cell.terminal_cell:    # No actions to take from a terminal cell.
      return best_actions 

    if col > 1 and self.get_cell(row, col - 1).value > cell.value:
      best_actions.append(Action.Left)
    
    if row > 1 and self.get_cell(row - 1, col).value > cell.value:
      best_actions.append(Action.Up)

    if row < self.rows and self.get_cell(row + 1, col).value > cell.value:
      best_actions.append(Action.Down)

    if col < self.cols and self.get_cell(row, col + 1).value > cell.value:
      best_actions.append(Action.Right)

    if len(best_actions) == 0: # If there's no best action then we can technically move anywhere.
      return [Action.Left, Action.Up, Action.Down, Action.Right]
    else:  
      return best_actions

  # Returns a random starting cell that's not a terminal cell
  def get_random_starting_cell(self):
    idx = random.randint(1, len(self.cells) - 2)
    return self.cells[idx]

  # Return all of the available actions from a particular state (position)
  def available_actions(self, row, col):
    return self.get_cell(row, col).available_actions

  # Reset all the cell values back to their original value
  def reset_values(self):
    for i in range(0, len(self.cells)):
      self.cells[i].value = 0

  # Gets a new cell based on a state + action
  def cell_from_action(self, current_cell, action):
    if action == None:
      return current_cell 
    if action == Action.Up:
      return self.get_cell(current_cell.row - 1, current_cell.col)
    if action == Action.Down:
      return self.get_cell(current_cell.row + 1, current_cell.col)
    if action == Action.Left:
      return self.get_cell(current_cell.row, current_cell.col - 1)
    if action == Action.Right:
      return self.get_cell(current_cell.row, current_cell.col + 1)

  # Returns a clean-looking gridview output in HTML format so that we can visualize the state of a grid at any point.
  def get_grid(self, k, optimal):
    idx = 0    
    
    html = '<table><tr><td>&nbsp;</td><td style="text-align: center">Vk for the MC policy</td><td width="25">&nbsp;</td><td style="text-align: center;">Current Policy w.r.t. Vk</td><td>&nbsp;</td><td style="text-align: center;">Optimal Policy w.r.t. Vk</td></tr>'
    html += '<tr>'
    html += '<td width="75"><i>k = {0}</i></td>'.format(k)

    # Vk for Policy
    html += '<td>'
    html += '<table style="border: 1px solid black; border-collapse: collapse;">'
    for r in range(1, self.rows + 1):    
      html += '<tr>'  
      for c in range(1, self.cols + 1):
        html += '<td style="height: 45px; width: 45px; text-align: center; border: 1px solid black;">{0}</td>'.format(self.cells[idx].value)
        idx += 1
      html += '</tr>'  
    html += '</table>'
    html += '</td>'
    html += '<td width="25">&nbsp;</td>'
        
    # Print out the current policy
    html += '<td>'
    html += '<table style="border: 1px solid black; border-collapse: collapse;">'
    idx = 0
    for r in range(1, self.rows + 1):    
      html += '<tr>'  
      for c in range(1, self.cols + 1):
        color = ''
        if self.get_cell(r, c).terminal_cell:
          color = ' background-color: #C8C8C8;' # Grey
        elif self.get_arrows(self.get_cell_best_actions(r, c)) == self.get_arrows(optimal[idx]):
          color = ' background-color: #33CC33;' # Green 
        else:
          color = ' background-color: #FF0000;' # Red 
        html += '<td style="height: 45px; width: 45px; text-align: center; border: 1px solid black;{0}">{1}</td>'.format(color, self.get_arrows(self.get_cell_best_actions(r, c)))
        idx += 1
      html += '</tr>'  
    html += '</table>'
    html += '</td>'
    html += '<td width="25">&nbsp;</td>'
    
    # Print out the optimal policy
    html += '<td>'
    html += '<table style="border: 1px solid black; border-collapse: collapse;">'
    idx = 0
    for r in range(1, self.rows + 1):    
      html += '<tr>'  
      for c in range(1, self.cols + 1):
        if idx <= len(optimal) - 1:
          color = ''
          if self.get_cell(r, c).terminal_cell:
            color = ' background-color: #C8C8C8;' # Grey
          html += '<td style="height: 45px; width: 45px; text-align: center; border: 1px solid black;{0}">{1}</td>'.format(color, self.get_arrows(optimal[idx]))
        else:
          html += '<td style="height: 45px; width: 45px; text-align: center; border: 1px solid black;">{0}</td>'.format('??')
        idx += 1
      html += '</tr>'  
    html += '</table>'
    html += '</td>'
    
    html += '</tr>'
    html += '</table>' 
    html += '<br/>' 
    return html 

  # Helper method to get arrows for the gridworld view
  def get_arrows(self, actions):
    html = ''

    if Action.Left in actions:
      html += '&#8592;'
    
    if Action.Up in actions:
      html += '&#8593;'

    if Action.Down in actions:
      html += '&#8595;'
    
    if Action.Right in actions:
      html += '&#8594;'

    return html 

In [53]:
# My elipson greedy policy implementation
class Elipson_Greedy_Policy(object):
  def __init__(self, Q, Q_updates, e):
    self.Q = Q
    self.Q_updates = Q_updates
    self.e = e

  def select_action(self, state):
    r = random.random()
    if r > self.e:
      action_Q = self.Q[state.index] 
      choices = [] 
      best_val = -10000000
      
      for a, value in action_Q.items():        
        if value > best_val:
          choices = []
          choices.append(Action(a))
          best_val = value 
        elif value == best_val:
          choices.append(Action(a))          
      return random.choice(choices)
    else:
      return random.choice([Action.Up, Action.Down, Action.Left, Action.Right])

  def get_q_value(self, St, At):
    return self.Q[St.index][int(At)]

  def get_q_updates(self, St, At):
    return self.Q_updates[St.index][int(At)]

  # Updating the Q value that this policy is using
  def update_Q(self, St, At, value):
    self.Q[St.index][At] = value 
    self.Q_updates[St.index][At] += 1

# Binds a gridworld and policy together so that an episode can be generated with any policy
class Episode(object):
  def __init__(self, grid, start_row, start_col, policy):
    self.grid = grid 
    self.start_row = start_row 
    self.start_col = start_col
    self.policy = policy
    self.actions = []
    self.states = []
    self.rewards = []

  # Generates an update and records all states and actions taken, following the injected policy
  def generate(self):  
    current_cell = self.grid.get_cell(self.start_row, self.start_col)
    self.states.append(current_cell)

    t = 0
    while not current_cell.terminal_cell and t <= 50:   # If we go over 50, we're probably in an infinite loop
      action = policy.select_action(current_cell)            
      self.actions.append(action)

      if action in current_cell.available_actions:
        current_cell = grid.cell_from_action(current_cell, action)
      self.states.append(current_cell)
      self.rewards.append(-1)  # No reward if we're not on a terminal state
      t +=1
    self.rewards.append(-1)     # Terminal state reward still costs a movement point

# Pretty Print for the Q state table and the optimal pi table.
def pretty_print_Q_table(grid, policy):  
  html = '<br/><table><tr><td><b>Q State Table</b></td><td>&nbsp;</td><td><b>Optimal pi</b></td></tr>'
  html += '<tr><td valign="top">'
  
  # Q state table
  html += '<table style="border: 1px solid black; border-collapse: collapse;">'   
  html += '<tr><td style="border: 1px solid black; padding:5px;"><b>State (R:C)</b></td><td style="border: 1px solid black; padding:5px;"><b>Action</b></td><td style="border: 1px solid black; padding:5px;"><b>Q*(s,a)</b></td><td style="border: 1px solid black; padding:5px;"><b>Q Updates</b></td></tr>'
  for cell in grid.cells:
    html += '<tr><td style="border: 1px solid black; padding:5px;">{0}</td><td style="border: 1px solid black; padding:5px;">U</td><td style="border: 1px solid black; padding:5px;">{1}</td><td style="border: 1px solid black; padding:5px;">{2}</td></tr>'.format(cell, policy.get_q_value(cell, Action.Up), policy.get_q_updates(cell, Action.Up))
    html += '<tr><td style="border: 1px solid black; padding:5px;">{0}</td><td style="border: 1px solid black; padding:5px;">D</td><td style="border: 1px solid black; padding:5px;">{1}</td><td style="border: 1px solid black; padding:5px;">{2}</td></tr>'.format(cell, policy.get_q_value(cell, Action.Down), policy.get_q_updates(cell, Action.Down))
    html += '<tr><td style="border: 1px solid black; padding:5px;">{0}</td><td style="border: 1px solid black; padding:5px;">L</td><td style="border: 1px solid black; padding:5px;">{1}</td><td style="border: 1px solid black; padding:5px;">{2}</td></tr>'.format(cell, policy.get_q_value(cell, Action.Left), policy.get_q_updates(cell, Action.Left))
    html += '<tr><td style="border: 1px solid black; padding:5px;">{0}</td><td style="border: 1px solid black; padding:5px;">R</td><td style="border: 1px solid black; padding:5px;">{1}</td><td style="border: 1px solid black; padding:5px;">{2}</td></tr>'.format(cell, policy.get_q_value(cell, Action.Right), policy.get_q_updates(cell, Action.Right))
  html += '</table></td>'   

  html += '<td>&nbsp;</td>'

  # Optimal pi
  html += '<td valign="top"><table style="border: 1px solid black; border-collapse: collapse;">'  
  html += '<tr><td style="border: 1px solid black; padding:5px;"><b>State (R:C)</b></td><td style="border: 1px solid black; padding:5px;"><b>Optimal pi</b></td></tr>' 
  for cell in grid.cells:
    pi = grid.get_cell_best_actions(cell.row, cell.col)
    friendly = ''
    if not pi == None:
      if Action.Up in pi:
        friendly += 'U,'
      if Action.Down in pi:
        friendly += 'D,'
      if Action.Left in pi:
        friendly += 'L,'
      if Action.Right in pi:
        friendly += 'R,'
      friendly = friendly[:-1] # Strip off final comma

    html += '</td><tr><td style="border: 1px solid black; padding:5px;">{0}</td><td style="border: 1px solid black; padding:5px;">{1}</td></tr>'.format(cell, friendly)
  html += '</table>'

  return html   

In [57]:
# Monte Carlo Policy Optimization

# Helper function to calculate the average return, given a state and action.
def avg_return(returns, St, At):
  cnt = 0
  sm = 0.0
  
  for v in returns[St.index][At]:
    cnt += 1    
    sm += v
    
  return (sm / cnt)

# Helper function to calculate the average Q value, given a state. 
def max_Q_state_value(Q, St):
  best_val = -10000000

  for key, value in Q[St].items():
    if value > best_val:
      best_val = value

  return round(best_val, 1)

# Helper method to determine whether or not a State/Action pair appears earlier in an episode
def appears_earlier(St, At, episode, Ct):
  for t in range(0, Ct):    
    if St == episode.states[t] and At == episode.actions[t]:      
      return True

  return False  

# What are the terminal cells for this gridworld?
terminal_cells = [Cell(1, 1, 0, True), Cell(4, 4, 15, True)]

# What are the optimal actions for each cell?
optimal = [[], [Action.Left], [Action.Left], [Action.Left, Action.Down],
           [Action.Up], [Action.Left, Action.Up], [Action.Left, Action.Up, Action.Down, Action.Right], [Action.Down],
           [Action.Up], [Action.Left, Action.Up, Action.Down, Action.Right], [Action.Down, Action.Right], [Action.Down],
           [Action.Up, Action.Right], [Action.Right], [Action.Right], []]

# Create the gridworld 
grid = Grid(4, 4, terminal_cells)
grid.reset_values()

# Initialize our Q and our Returns
#
# Q will be initialized with a set for each state and a sub-set for each action, initialized to 0
# Returns will be initialized to a set containing all of the states, each containing an empty list of values
Q = {}
Q_updates = {}
Returns = {}

for cell in grid.cells:
  u = {}
  a = {}
  r = {}
  available_actions = [Action.Up, Action.Down, Action.Left, Action.Right]

  for action in available_actions:
    if cell.terminal_cell:
      a[int(action)] = 0
    else:  
      a[int(action)] = -1
    u[int(action)] = 0
    r[int(action)] = []

  Q[cell.index] = a  
  Q_updates[cell.index] = u
  Returns[cell.index] = r  

# Create the elipson greedy policy we want to use
policy = Elipson_Greedy_Policy(Q, Q_updates, e=0.1)  # 10% exploration
html = grid.get_grid(0, optimal)  # Get a displayable grid for k = 0

# Optimization parameters
NUM_ITERATIONS = 50000  # k
KEY_K = [1, 2, 3, 10, 50, 500, 1000, 5000, 10000, 50000, 100000]   # Which k values to display in our output
gamma = 1  # A discount factor that I've tuned 

# Run a number of episodes
for k in range(1, NUM_ITERATIONS + 1):      # For each episode
  start = grid.cells[(k % 15)]              # Uniformly pull a starting position  
  
  episode = Episode(grid, start.row, start.col, policy) # Create a new episode and inject our elipson greedy policy
  episode.generate() # Run the episode using our elipson greedy policy
  G = 0  
  
  for t in range(len(episode.states) - 2, -1, -1):   # Loop for each step of episode, t = T - 1, T - 2,...,0  
    G = (gamma * G) + episode.rewards[t + 1]         # Set G to gamma + (Rt + 1)
    St = episode.states[t]    # State at this step
    At = episode.actions[t]   # Action at this step

    if not appears_earlier(St, At, episode, t): # Unless the pair St,At appears in S0,A0,S1,A1...,St-1,At-1    
       Returns[St.index][At].append(G)    # Append G to Returns(St, At)
       avg = avg_return(Returns, St, At)  # Calculate the average return for Q
       policy.update_Q(St, At, avg)       # Update the Q value

  # Update each cell in the grid with the average Q value for that state.
  for cell in grid.cells:
    if not cell.terminal_cell:
      aQ = max_Q_state_value(Q, cell.index)
      cell.value = aQ

  # Display the grid in the output (if this is a key iteration)
  if k in KEY_K:
    html += grid.get_grid(k, optimal);

# Get the final tables that instructor wants to see.
html += pretty_print_Q_table(grid, policy)

# If this doesn't work, try IPython.display.HTML('<img src="/content/elmo_1.jpg">') and then re-run (weird colab bug)
IPython.display.HTML(html)